Credit Card Fraud Detection Project

In [13]:
import IPython
IPython.display.clear_output(wait=True)

In [12]:
import os
import glob

# Define the file types to delete (adjust extensions if needed)
files_to_delete = ["*.pkl", "*.csv", "*.xlsx"]  # Adjust as needed

# Loop through each file type and delete them
for file_type in files_to_delete:
    for file in glob.glob(file_type):  # Find all matching files
        try:
            os.remove(file)  # Delete the file
            print(f"Deleted: {file}")
        except Exception as e:
            print(f"Error deleting {file}: {e}")

print("\n✅ All saved models, scalers, and files have been deleted!")


Deleted: best_xgb_model.pkl
Deleted: final_log_model.pkl
Deleted: final_rf_model.pkl
Deleted: final_threshold.pkl
Deleted: final_xgb_model.pkl
Deleted: logistic_regression_model.pkl
Deleted: random_forest_model.pkl
Deleted: scaler.pkl
Deleted: xgb_initial_model.pkl
Deleted: fraud_predictions_check.xlsx

✅ All saved models, scalers, and files have been deleted!


In [1]:
# Data Handling
import pandas as pd
import numpy as np

# Data Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Machine Learning Models
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# Model Training & Hyperparameter Tuning
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

# Model Evaluation Metrics
from sklearn.metrics import (
    precision_score, recall_score, f1_score, average_precision_score, precision_recall_curve
)
from sklearn.metrics import classification_report


# Ensemble Learning
from sklearn.ensemble import VotingClassifier

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Save the model
import joblib
import json

# analyze the impact of each feature on the model's decision-making process.
import shap

Load and Explore the Dataset (Exploratory Data Analysis (EDA)

In [2]:
# Load dataset

credit_card_fraud = pd.read_csv("C:\\Users\\USER\\Downloads\\creditcard.csv")

In [3]:
# Display the first few rows

credit_card_fraud.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [4]:
# Display the last few rows

credit_card_fraud.tail()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,...,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0
284806,172792.0,-0.533413,-0.189733,0.703337,-0.506271,-0.012546,-0.649617,1.577006,-0.414650,0.486180,...,0.261057,0.643078,0.376777,0.008797,-0.473649,-0.818267,-0.002415,0.013649,217.00,0


In [5]:
# Check dataset shape

credit_card_fraud.shape

(284807, 31)

In [6]:
# Check class distribution

credit_card_fraud["Class"].value_counts()

Class
0    284315
1       492
Name: count, dtype: int64

In [7]:
# exploring the datatype of each column

credit_card_fraud.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [8]:
# Check for missing values

credit_card_fraud.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

In [9]:
# Check basic statistics of the dataset

credit_card_fraud.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,284807.000000,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,...,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,284807.000000,284807.000000
mean,94813.859575,1.168375e-15,3.416908e-16,-1.379537e-15,2.074095e-15,9.604066e-16,1.487313e-15,-5.556467e-16,1.213481e-16,-2.406331e-15,...,1.654067e-16,-3.568593e-16,2.578648e-16,4.473266e-15,5.340915e-16,1.683437e-15,-3.660091e-16,-1.227390e-16,88.349619,0.001727
std,47488.145955,1.958696e+00,1.651309e+00,1.516255e+00,1.415869e+00,1.380247e+00,1.332271e+00,1.237094e+00,1.194353e+00,1.098632e+00,...,7.345240e-01,7.257016e-01,6.244603e-01,6.056471e-01,5.212781e-01,4.822270e-01,4.036325e-01,3.300833e-01,250.120109,0.041527
min,0.000000,-5.640751e+01,-7.271573e+01,-4.832559e+01,-5.683171e+00,-1.137433e+02,-2.616051e+01,-4.355724e+01,-7.321672e+01,-1.343407e+01,...,-3.483038e+01,-1.093314e+01,-4.480774e+01,-2.836627e+00,-1.029540e+01,-2.604551e+00,-2.256568e+01,-1.543008e+01,0.000000,0.000000
25%,54201.500000,-9.203734e-01,-5.985499e-01,-8.903648e-01,-8.486401e-01,-6.915971e-01,-7.682956e-01,-5.540759e-01,-2.086297e-01,-6.430976e-01,...,-2.283949e-01,-5.423504e-01,-1.618463e-01,-3.545861e-01,-3.171451e-01,-3.269839e-01,-7.083953e-02,-5.295979e-02,5.600000,0.000000
50%,84692.000000,1.810880e-02,6.548556e-02,1.798463e-01,-1.984653e-02,-5.433583e-02,-2.741871e-01,4.010308e-02,2.235804e-02,-5.142873e-02,...,-2.945017e-02,6.781943e-03,-1.119293e-02,4.097606e-02,1.659350e-02,-5.213911e-02,1.342146e-03,1.124383e-02,22.000000,0.000000
75%,139320.500000,1.315642e+00,8.037239e-01,1.027196e+00,7.433413e-01,6.119264e-01,3.985649e-01,5.704361e-01,3.273459e-01,5.971390e-01,...,1.863772e-01,5.285536e-01,1.476421e-01,4.395266e-01,3.507156e-01,2.409522e-01,9.104512e-02,7.827995e-02,77.165000,0.000000
max,172792.000000,2.454930e+00,2.205773e+01,9.382558e+00,1.687534e+01,3.480167e+01,7.330163e+01,1.205895e+02,2.000721e+01,1.559499e+01,...,2.720284e+01,1.050309e+01,2.252841e+01,4.584549e+00,7.519589e+00,3.517346e+00,3.161220e+01,3.384781e+01,25691.160000,1.000000


In [10]:
# imbalance ratio of fruad cases and normal cases

fraud_cases = credit_card_fraud[credit_card_fraud["Class"] == 1]
normal_cases = credit_card_fraud[credit_card_fraud["Class"] == 0]

imbalance_ratio = len(normal_cases) / len(fraud_cases)
print(f"Imbalance Ratio: {imbalance_ratio:.4f}")

Imbalance Ratio: 577.8760


Split Features & Target

We separate the features from the target variable and split the data into training and testing sets.

In [11]:
X = credit_card_fraud.drop(columns=['Class'])  # Adjust column name accordingly
y = credit_card_fraud['Class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [ ]:
# Randomly sample 10 records from the test set (before scaling)
sample_indices = X_test.sample(10, random_state=42).index
X_manual_sample = X_test.loc[sample_indices]
y_manual_sample = y_test.loc[sample_indices]

# Save the original (unscaled) test samples for manual checking
X_manual_sample.to_csv("manual_test_sample_raw.csv", index=False)


In [ ]:
# Ensure indexes of X and y are correctly aligned
assert X.index.equals(y.index), "Mismatch between X and y indexes!"


In [ ]:
# Ensure X and y are Pandas DataFrame/Series
X = pd.DataFrame(X)  # If X is a NumPy array, convert it to DataFrame
y = pd.Series(y)  # If y is a NumPy array, convert it to Series

# Perform train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Now check index alignment
assert X_train.index.equals(y_train.index), "Mismatch in X_train and y_train indexes!"
assert X_test.index.equals(y_test.index), "Mismatch in X_test and y_test indexes!"


In [ ]:
# Take a sample index
sample_idx = X_train.index[0]  

# Verify feature-label consistency
print("Features:", X_train.loc[sample_idx])
print("Label:", y_train.loc[sample_idx])


In [ ]:
sample_features = X_test.iloc[0].values.reshape(1, -1)  # Ensure correct shape
prediction = model.predict(sample_features)
print("Predicted Label:", prediction)


In [ ]:
print(0 in X_test.index)  # True if present, False if missing


In [ ]:
sample_index = X_test.index[0]  # Pick the first available index
sample_features = X_test.loc[sample_index].values.reshape(1, -1)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_train, X_test = X_train.reset_index(drop=True), X_test.reset_index(drop=True)
y_train, y_test = y_train.reset_index(drop=True), y_test.reset_index(drop=True)


In [ ]:
# Check index alignment for training set
assert X_train.index.equals(y_train.index), "Mismatch in X_train and y_train indexes!"

# Check index alignment for test set
assert X_test.index.equals(y_test.index), "Mismatch in X_test and y_test indexes!"


In [ ]:
X_test = pd.DataFrame(X_test, columns=feature_names)  # Ensure you have feature names

In [ ]:
X_test

In [ ]:
print(type(X_test))  # Should be a Pandas DataFrame
print(type(y_test))  # Should be a Pandas Series


In [ ]:
# # Reset index for consistency
# X_test = X_test.reset_index(drop=True)
# y_test = y_test.reset_index(drop=True)

# Now, select fraud and non-fraud indices
fraud_indices = y_test[y_test == 1].index.tolist()
non_fraud_indices = y_test[y_test == 0].index.tolist()

# Randomly select 10 fraud and non-fraud samples
num_fraud = min(10, len(fraud_indices))
num_non_fraud = min(10, len(non_fraud_indices))

random_fraud = np.random.choice(fraud_indices, size=num_fraud, replace=False).tolist()
random_non_fraud = np.random.choice(non_fraud_indices, size=num_non_fraud, replace=False).tolist()

# Combine indices
random_indices = random_fraud + random_non_fraud  

# Select samples
X_test_samples = X_test.loc[random_indices]
y_test_samples = y_test.loc[random_indices]

# Display results
print("Selected X_test Samples:\n", X_test_samples)
print("\nActual y_test Labels:\n", y_test_samples.value_counts())


In [ ]:
# # Get fraud and non-fraud indices
# fraud_indices = y_test[y_test == 1].index
# non_fraud_indices = y_test[y_test == 0].index

# # Determine how many fraud cases exist
# num_fraud = min(10, len(fraud_indices))  # Ensure we don't exceed available fraud cases
# num_non_fraud = min(10, len(non_fraud_indices))  # Same for non-fraud

# # Select random samples
# random_fraud = np.random.choice(fraud_indices, size=num_fraud, replace=False) if num_fraud > 0 else []
# random_non_fraud = np.random.choice(non_fraud_indices, size=num_non_fraud, replace=False)

# # Combine indices
# random_indices = np.concatenate([random_fraud, random_non_fraud])

# # Select corresponding X_test and y_test samples
# X_test_samples = X_test.loc[random_indices]
# y_test_samples = y_test.loc[random_indices]

# # Display results
# print("Selected X_test Samples:\n", X_test_samples)
# print("\nActual y_test Labels:\n", y_test_samples.value_counts())  # Show fraud (1) and non-fraud (0) count



import numpy as np
import pandas as pd

# Ensure X_test is a DataFrame with the same index as y_test
if isinstance(X_test, np.ndarray):  
    X_test = pd.DataFrame(X_test, index=y_test.index, columns=feature_names)  # Ensure the index matches y_test

# Get fraud and non-fraud indices
fraud_indices = y_test[y_test == 1].index
non_fraud_indices = y_test[y_test == 0].index

# Determine how many fraud cases exist
num_fraud = min(10, len(fraud_indices))  
num_non_fraud = min(10, len(non_fraud_indices))  

# Select random samples
random_fraud = np.random.choice(fraud_indices, size=num_fraud, replace=False) if num_fraud > 0 else []
random_non_fraud = np.random.choice(non_fraud_indices, size=num_non_fraud, replace=False)

# Convert NumPy arrays to lists
random_fraud = list(random_fraud)
random_non_fraud = list(random_non_fraud)

# Combine indices
random_indices = random_fraud + random_non_fraud  

# Select corresponding X_test and y_test samples
X_test_samples = X_test.loc[random_indices]  # Now, indices should exist in X_test
y_test_samples = y_test.loc[random_indices]

# Display results
print("Selected X_test Samples:\n", X_test_samples)
print("\nActual y_test Labels:\n", y_test_samples.value_counts())  



In [ ]:
X_test_samples

In [ ]:
y_test_samples


Feature Scaling

Since our features might have different scales, we standardize them.

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
print(X_train.shape)

In [ ]:
X_test

print(X_test.shape)

Train Individual Models

We train XGBoost, Random Forest, and Logistic Regression models.

In [ ]:
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
xgb_model.fit(X_train, y_train)

rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

log_model = LogisticRegression(solver='liblinear', random_state=42)
log_model.fit(X_train, y_train)

Evaluate Individual Models Before Tuning

In [ ]:
def evaluate_model(model, X_test, y_test, threshold=0.5):
    y_probs = model.predict_proba(X_test)[:, 1]
    y_pred = (y_probs >= threshold).astype(int)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc_pr = average_precision_score(y_test, y_probs)
    print(f'Precision: {precision:.4f}')
    print(f'Recall: {recall:.4f}')
    print(f'F1-Score: {f1:.4f}')
    print(f'AUC-PR: {auc_pr:.4f}')

print("\n📌 XGBoost Performance:")    
evaluate_model(xgb_model, X_test, y_test)

print("\n📌 Random Forest Performance:")
evaluate_model(rf_model, X_test, y_test)

print("\n📌 Logistic Regression Performance:")
evaluate_model(log_model, X_test, y_test)



Hyperparameter Tuning

We optimize the hyperparameters of the XGBoost model using GridSearchCV

In [ ]:
# Define hyperparameter grid
# param_grid_xgb = {
#     'n_estimators': [50, 100, 200],  
#     'max_depth': [3, 5, 7],  
#     'learning_rate': [0.01, 0.1, 0.2]  
# }

param_grid_xgb = {
    'n_estimators': [100, 200],  
    'max_depth': [5, 7],  
    'learning_rate': [0.05, 0.1],  
    'scale_pos_weight': [1, 10, 25, 50]  # ✅ Adjust for imbalance
    # 'scale_pos_weight': [10, 25, 50]
}


# ✅ Define XGBoost model with additional parameters for hyperparameter tuning
xgb_models = XGBClassifier(
    use_label_encoder=False, 
    objective="binary:logistic",   # ✅ Correct for fraud detection
    eval_metric=["logloss", "aucpr"],  # ✅ LogLoss + AUC-PR for imbalanced data
    tree_method="hist",   # ✅ Faster training
    verbosity=1,  # ✅ Shows progress logs
    random_state=42  
)

# Define scoring metrics
scoring_metrics = {
    "F1": "f1",
    "Precision": "precision",
    "Recall": "recall",
    "AUC-PR": "average_precision"  # ✅ Optimizing for AUC-PR
}

# Perform Grid Search
grid_xgb = GridSearchCV(
    xgb_models, param_grid_xgb, cv=3, scoring=scoring_metrics, 
    n_jobs=-1, refit="AUC-PR"  # ✅ Selects best model based on AUC-PR
)

# Fit GridSearchCV using the scaled training data
grid_xgb.fit(X_train, y_train)  # ✅ Use X_train_scaled

# Get the best model based on AUC-PR
best_xgb = grid_xgb.best_estimator_

# Display the best hyperparameters
print("Best Parameters:", grid_xgb.best_params_)

# ✅ Display best scores for all metrics correctly
print("\nBest Scores:")
for metric in scoring_metrics.keys():  # Loop over metric names
    metric_key = f"mean_test_{metric}"  # Correct key in cv_results_
    score = grid_xgb.cv_results_[metric_key].max()  # Get the best value for each metric
    print(f"{metric}: {score:.4f}")

Precision-Recall Curve & Optimal Threshold Selection

We compute precision-recall scores at different thresholds and select the best one automatically.

In [ ]:
# # Fine-Tune scale_pos_weight Further
# best_xgb.set_params(scale_pos_weight=5)  # Try 5, 10, or 15
# best_xgb.fit(X_train, y_train)

# Get fraud probabilities
y_probs = best_xgb.predict_proba(X_test)[:, 1]  

# Compute precision-recall curve
precisions, recalls, thresholds = precision_recall_curve(y_test, y_probs)

# Compute F1 scores, handling NaN cases
f1_scores = np.nan_to_num(2 * (precisions * recalls) / (precisions + recalls))

# Ensure threshold index is correctly aligned
best_threshold = thresholds[np.argmax(f1_scores[:-1])]  # Ensure no index mismatch

print(f"\n✅ Best Threshold: {best_threshold:.3f}")

# Evaluate Models with Best Threshold
print("\n📌 XGBoost Performance:")
evaluate_model(best_xgb, X_test, y_test, best_threshold)

print("\n📌 XGBoosts Performance:")
evaluate_model(xgb_model, X_test, y_test, best_threshold)

print("\n📌 Random Forest Performance:")
evaluate_model(rf_model, X_test, y_test, best_threshold)

print("\n📌 Logistic Regression Performance:")
evaluate_model(log_model, X_test, y_test, best_threshold)

Apply the Best Threshold to Your Initial XGBoost model

In [ ]:
# Get fraud probabilities from your initial XGBoost model
y_probs_initial = xgb_model.predict_proba(X_test)[:, 1]  

# Apply the best threshold from the tuned model
y_pred_initial = (y_probs_initial >= best_threshold).astype(int)

# Evaluate initial model with the optimized threshold
print("\n📌 Initial XGBoost Model Performance with Best Threshold:")
print(classification_report(y_test, y_pred_initial))

In [ ]:
# # Plot Precision-Recall vs Threshold
# plt.figure(figsize=(8, 6))
# plt.plot(best_threshold, precisions[:-1], label="Precision", linestyle="--")
# plt.plot(best_threshold, recalls[:-1], label="Recall")
# plt.xlabel("best_threshold")
# plt.ylabel("Score")
# plt.title("Precision-Recall vs. Decision Threshold")
# plt.legend()
# plt.grid()
# plt.show()


# Plot Precision-Recall vs Threshold
plt.figure(figsize=(8, 6))
plt.plot(thresholds, precisions[:-1], label="Precision", linestyle="--")
plt.plot(thresholds, recalls[:-1], label="Recall")
plt.axvline(best_threshold, color='r', linestyle=":", label="Best Threshold")  # Mark the best threshold
plt.xlabel("Decision Threshold")
plt.ylabel("Score")
plt.title("Precision-Recall vs. Decision Threshold")
plt.legend()
plt.grid()
plt.show()




Building an Ensemble Model (XGBoost + RF + Logistic)

In [ ]:
# ensemble = VotingClassifier(estimators=[('xgb', best_xgb), ('rf', rf_model), ('log', log_model)], voting='soft')
# ensemble.fit(X_train, y_train)
# evaluate_model(ensemble, X_test, y_test, best_threshold)

# # ensemble_model = VotingClassifier(
# #     estimators=[
# #         ('xgb', best_xgb),
# #         ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
# #         ('log', LogisticRegression())
# #     ],
# #     voting='soft',  # Probability-based voting
# #     weights=[3, 2, 1]  # Giving more weight to XGBoost
# # )
# # ensemble_model.fit(X_train, y_train)

🔹 SHAP Analysis for Feature Importance

In [ ]:
# XGBoost Model (Tree-Based)
explainer_xgb = shap.Explainer(best_xgb)
shap_values_xgb = explainer_xgb(X_test)
shap.summary_plot(shap_values_xgb, X_test)

In [ ]:
# XGBoosts Model (Tree-Based)
explainer_xgb = shap.Explainer(xgb_model)
shap_values_xgb_ = explainer_xgb(X_test)
shap.summary_plot(shap_values_xgb_, X_test)

In [ ]:
# # Random Forest Model (Tree-Based)
# explainer_rf = shap.TreeExplainer(rf_model)  # Use TreeExplainer for RF
# shap_values_rf = explainer_rf.shap_values(X_test)
# shap.summary_plot(shap_values_rf[1], X_test)  # Use class index [1] for fraud cases
# shap.summary_plot(shap_values_rf[0], X_test)


# Use the proper SHAP explainer for Random Forest
explainer_rf = shap.TreeExplainer(rf_model, feature_perturbation="tree_path_dependent")
shap_values_rf = explainer_rf.shap_values(X_test)

# Check if SHAP returns a list or array
if isinstance(shap_values_rf, list):  # Multi-class case
    shap.summary_plot(shap_values_rf[1], X_test)  # Fraud class
else:  # Single output case
    shap.summary_plot(shap_values_rf, X_test)  



print(type(shap_values_rf))  # Check if it's a list or NumPy array



print(np.shape(shap_values_rf))  # Print shape of SHAP values

In [ ]:

# Logistic Regression Model (Linear)
explainer_log = shap.LinearExplainer(log_model, X_train)  # Use LinearExplainer for LR
shap_values_log = explainer_log.shap_values(X_test)
shap.summary_plot(shap_values_log, X_test)

Run the Final Model Evaluation Check

In [ ]:
# Function to evaluate models and return key metrics
def evaluate_and_print(model, X_test, y_test, threshold, model_name):
    # Get predicted probabilities
    y_probs = model.predict_proba(X_test)[:, 1]
    
    # Apply best threshold
    y_pred = (y_probs >= threshold).astype(int)

    # Get evaluation metrics
    report = classification_report(y_test, y_pred, output_dict=True)
    
    # Print classification report
    print(f"\n📌 {model_name} Performance:")
    print(classification_report(y_test, y_pred))
    
    # Return F1-score for comparison
    return report["1"]["f1-score"]

# Define models
models = {
    "XGBoost (Tuned)": best_xgb,
    "XGBoost (Initial)": xgb_model,
    "Random Forest": rf_model,
    "Logistic Regression": log_model
}

# Store F1-scores for model selection
f1_scores = {}

# Best threshold found earlier
for name, model in models.items():
    f1_scores[name] = evaluate_and_print(model, X_test, y_test, best_threshold, name)

# Select the best model based on F1-score
best_model_name = max(f1_scores, key=f1_scores.get)
best_model = models[best_model_name]

print(f"\n🏆 Best Model for Deployment: {best_model_name} (F1-Score: {f1_scores[best_model_name]:.4f}) ✅")


In [ ]:
# Function to evaluate models and return key metrics with both thresholds
def evaluate_and_print(model, X_test_scaled, y_test, threshold, model_name, is_best_threshold=True):
    """
    This function evaluates the model using a specified threshold (either best or default).
    - model: The model to evaluate (e.g., XGBoost, Random Forest, Logistic Regression)
    - X_test_scaled: The test data for prediction
    - y_test: The true labels for the test data
    - threshold: The threshold to apply for classification (best or default)
    - model_name: The name of the model (for display)
    - is_best_threshold: Whether to use the best threshold or the default threshold (0.5)
    """
    # Get predicted probabilities
    y_probs = model.predict_proba(X_test_scaled)[:, 1]
    
    # Apply the appropriate threshold: either best threshold or default (0.5)
    threshold_to_use = threshold if is_best_threshold else 0.5
    y_pred = (y_probs >= threshold_to_use).astype(int)  # Convert probabilities to binary predictions
    
    # Get evaluation metrics
    report = classification_report(y_test, y_pred, output_dict=True)
    
    # Print classification report
    print(f"\n📌 {model_name} Performance (Threshold {threshold_to_use}):")
    print(classification_report(y_test, y_pred))
    
    # Return F1-score for comparison
    return report["1"]["f1-score"]

# Define models
models = {
    "XGBoost (Tuned)": best_xgb,
    "Random Forest": rf_model,
    "Logistic Regression": log_model
}

# Store F1-scores for model selection with both thresholds
f1_scores_default = {}
f1_scores_best = {}

# Evaluate each model with both the default and best threshold
for name, model in models.items():
    f1_scores_default[name] = evaluate_and_print(model, X_test_scaled, y_test, best_threshold, name, is_best_threshold=True)  # Using best threshold
    f1_scores_best[name] = evaluate_and_print(model, X_test_scaled, y_test, best_threshold, name, is_best_threshold=False)  # Using default threshold

# Compare the F1-scores and select the best model based on both thresholds
best_model_default_threshold = max(f1_scores_default, key=f1_scores_default.get)
best_model_best_threshold = max(f1_scores_best, key=f1_scores_best.get)

print(f"\n🏆 Best Model for Deployment (Default Threshold): {best_model_default_threshold} (F1-Score: {f1_scores_default[best_model_default_threshold]:.4f}) ✅")
print(f"🏆 Best Model for Deployment (Best Threshold): {best_model_best_threshold} (F1-Score: {f1_scores_best[best_model_best_threshold]:.4f}) ✅")


Save Models

In [ ]:
# Save models
joblib.dump(best_xgb, "best_xgb_model.pkl")
joblib.dump(xgb_model, "xgb_initial_model.pkl")
joblib.dump(rf_model, "random_forest_model.pkl")
joblib.dump(log_model, "logistic_regression_model.pkl")

# Save the fitted scaler
joblib.dump(scaler, "scaler.pkl")

# Convert best_threshold to a standard Python float before saving
best_threshold = float(best_threshold)  # Ensures JSON compatibility

# Save best threshold
with open("best_threshold.json", "w") as f:
    json.dump({"best_threshold": best_threshold}, f)

# Save feature names
feature_names = X.columns.tolist()
with open("feature_names.json", "w") as f:
    json.dump(feature_names, f)

print("All models, scaler, best threshold, and feature names have been successfully saved!")

Manual Check Process for model validation

Check the Files in my Directory(If models are saved)

In [ ]:
import os

# List all files in the current directory
print(os.listdir())

Load and Check the Saved Models

In [ ]:
# Load the models
loaded_best_xgb = joblib.load("best_xgb_model.pkl")
loaded_xgb = joblib.load("xgb_initial_model.pkl")
loaded_rf = joblib.load("random_forest_model.pkl")
loaded_log = joblib.load("logistic_regression_model.pkl")

# Load the scaler
loaded_scaler = joblib.load("scaler.pkl")

# Load the best threshold
with open("best_threshold.json", "r") as f:
    best_threshold_data = json.load(f)
    loaded_best_threshold = best_threshold_data["best_threshold"]

# Load feature names
with open("feature_names.json", "r") as f:
    loaded_feature_names = json.load(f)

# Print confirmations
print("✅ Models and scaler loaded successfully!")
print("✅ Best threshold:", loaded_best_threshold)
print("✅ Feature names:", loaded_feature_names[:5], "...")  # Print first 5 features


In [ ]:
# # Function to evaluate models and show actual vs. predicted values
# def check_predictions(model, X_test_samples, y_test_samples, best_threshold, model_name):
#     # Get predicted probabilities
#     y_probs = model.predict_proba(X_test_samples)[:, 1]

#     # Apply best threshold
#     y_pred = (y_probs >= best_threshold).astype(int)

#     # Return predicted values
#     return y_pred

# # Define models to check
# models = {
#     "XGBoost (Tuned)": best_xgb,
#     "XGBoost (Initial)": xgb_model,
#     "Random Forest": rf_model,
#     "Logistic Regression": log_model
# }

# # Store predictions
# predictions = {}

# for name, model in models.items():
#     predictions[name] = check_predictions(model, X_test_samples, y_test_samples, best_threshold, name)

# # Create results DataFrame
# results_df = pd.DataFrame({
#     "Actual": y_test_samples.values
# })

# # Add model predictions
# for name, preds in predictions.items():
#     results_df[name] = preds

# # Add a column to check if the predictions were correct for each model
# for name in models.keys():
#     results_df[f"{name} Correct?"] = (results_df["Actual"] == results_df[name]).astype(int)

# # Display results before saving
# print(results_df)

# # Save to Excel if needed
# output_file = "fraud_predictions_check.xlsx"
# results_df.to_excel(output_file, index=False)

# print(f"\n✅ Results saved to {output_file}")


In [ ]:
# Function to evaluate models and show actual vs. predicted values
def check_predictions(model, X_test_samples, y_test_samples, best_threshold, model_name, scaler):
    # Apply the same scaling transformation used during training
    X_test_scaled = scaler.transform(X_test_samples)

    # Get predicted probabilities
    y_probs = model.predict_proba(X_test_scaled)[:, 1]

    # Apply best threshold
    y_pred = (y_probs >= best_threshold).astype(int)

    # Return predicted values
    return y_pred

# Load the pre-trained scaler
loaded_scaler = joblib.load("scaler.pkl")

# Define models to check
models = {
    "XGBoost (Tuned)": best_xgb,
    "XGBoost (Initial)": xgb_model,
    "Random Forest": rf_model,
    "Logistic Regression": log_model
}

# Store predictions
predictions = {}

for name, model in models.items():
    predictions[name] = check_predictions(model, X_test_samples, y_test_samples, best_threshold, name, loaded_scaler)

# Create results DataFrame
results_df = pd.DataFrame({
    "Actual": y_test_samples.values
})

# Add model predictions
for name, preds in predictions.items():
    results_df[name] = preds

# Add a column to check if the predictions were correct for each model
for name in models.keys():
    results_df[f"{name} Correct?"] = (results_df["Actual"] == results_df[name]).astype(int)

# Display results before saving
print(results_df)

# Save to Excel if needed
output_file = "fraud_predictions_check.xlsx"
results_df.to_excel(output_file, index=False)

print(f"\n✅ Results saved to {output_file}")


In [ ]:
# import pandas as pd

# # Function to evaluate models and show actual vs. predicted values
# def check_predictions(model, X_test_samples):
#     # Get predicted probabilities
#     y_probs = model.predict_proba(X_test_samples)[:, 1]

#     # Apply best threshold
#     y_pred = (y_probs >= best_threshold).astype(int)

#     # Return predicted values
#     return y_pred

# # Define models to check
# models = {
#     "XGBoost (Tuned)": best_xgb,
#     "XGBoost (Initial)": xgb_model,
#     "Random Forest": rf_model,
#     "Logistic Regression": log_model
# }

# # Store predictions
# predictions = {}

# for name, model in models.items():
#     predictions[name] = check_predictions(model, X_test_samples)

# # Create results DataFrame
# results_df = pd.DataFrame({
#     "Actual": y_test_samples.values
# })

# # Add model predictions
# for name, preds in predictions.items():
#     results_df[name] = preds

# # Add a column to check if the predictions were correct for each model
# for name in models.keys():
#     results_df[f"{name} Correct?"] = (results_df["Actual"] == results_df[name]).astype(int)

# # Display results before saving
# print(results_df.head())  # Show only the first few rows for better readability

# # Save to Excel
# output_file = "fraud_predictions_check.xlsx"
# results_df.to_excel(output_file, index=False)

# print(f"\n✅ Results saved to {output_file}")


In [ ]:
misclassified = results_df[results_df["Actual"] != results_df["XGBoost (Tuned)"]]
print(misclassified)


In [ ]:
misclassified_probs = best_xgb.predict_proba(X_test_samples.iloc[misclassified.index])[:, 1]
misclassified["Predicted Probabilities"] = misclassified_probs
print(misclassified)


In [ ]:
new_threshold = 0.8
new_preds = (best_xgb.predict_proba(X_test_samples)[:, 1] >= new_threshold).astype(int)

# Check new misclassified cases
new_misclassified = results_df[results_df["Actual"] != new_preds]
print(new_misclassified)


In [ ]:
print("X_test_samples Index:\n", X_test_samples.index[:10])  # Show first 10 indices
print("results_df Index:\n", results_df.index[:10])  # Show first 10 indices


In [ ]:
results_df = results_df.set_index(X_test_samples.index)


In [ ]:
correct_fraud_cases = X_test_samples.loc[
    (results_df["Actual"] == 1) & (results_df["XGBoost (Tuned) Correct?"] == 1)
]


In [ ]:
# Reindex results_df to match X_test_samples
aligned_results_df = results_df.loc[X_test_samples.index]

# Now apply the boolean indexing with aligned indices
correct_fraud_cases = X_test_samples.loc[
    (aligned_results_df["Actual"] == 1) & (aligned_results_df["XGBoost (Tuned) Correct?"] == 1)
]

# Check results
print(correct_fraud_cases.head())


In [ ]:
# Get misclassified features using .iloc (by position)
misclassified_features = X_test_samples.iloc[misclassified.index]

# Get correctly classified fraud cases using .loc (boolean indexing)
correct_fraud_cases = X_test_samples.loc[
    (results_df["Actual"] == 1) & (results_df["XGBoost (Tuned) Correct?"] == 1)
]

# Compare statistics
print("Misclassified Fraud Case Stats:\n", misclassified_features.describe())
print("\nCorrectly Classified Fraud Case Stats:\n", correct_fraud_cases.describe())


In [ ]:
print("Correctly Classified Fraud Cases:\n", correct_fraud_cases)
print("Shape:", correct_fraud_cases.shape)  # Check how many cases were found


In [ ]:
# Display misclassified fraud cases
print("Misclassified Fraud Cases:\n", misclassified_features)

# Display statistics of misclassified fraud cases
print("\nMisclassified Fraud Case Stats:\n", misclassified_features.describe())

# Compare with correctly classified fraud cases
print("\nCorrectly Classified Fraud Case Stats:\n", correct_fraud_cases.describe())


In [ ]:
# Get probability predictions
misclassified_probs = model.predict_proba(misclassified_features)

# Display probability scores
print("Misclassified Case Probabilities:\n", misclassified_probs)


In [ ]:
new_threshold = 0.3  # Adjust this value
y_probs = model.predict_proba(X_test)[:, 1]  # Get fraud probabilities
y_pred_new = (y_probs >= new_threshold).astype(int)

# Evaluate new threshold
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_new))


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Get feature importances (works for tree-based models like XGBoost or RF)
feature_importances = model.feature_importances_
features = X_test.columns

# Sort and plot top features
sorted_idx = np.argsort(feature_importances)[::-1]
plt.figure(figsize=(10, 6))
plt.barh(features[sorted_idx][:10], feature_importances[sorted_idx][:10])
plt.xlabel("Feature Importance Score")
plt.ylabel("Feature")
plt.title("Top 10 Important Features")
plt.gca().invert_yaxis()
plt.show()


In [ ]:
scale_pos_weight = len(y_train[y_train == 0]) / len(y_train[y_train == 1])  # Compute weight
model = XGBClassifier(scale_pos_weight=scale_pos_weight)
model.fit(X_train, y_train)


In [ ]:
scale_pos_weight = len(y_train[y_train == 0]) / len(y_train[y_train == 1])
model = XGBClassifier(scale_pos_weight=scale_pos_weight)
model.fit(X_train, y_train)


In [ ]:
# import matplotlib.pyplot as plt
# import numpy as np

# Get feature importances (only for tree-based models)
feature_importances = model.feature_importances_
features = X_test.columns

# Sort and plot top features
sorted_idx = np.argsort(feature_importances)[::-1]
plt.figure(figsize=(10, 6))
plt.barh(features[sorted_idx][:10], feature_importances[sorted_idx][:10])
plt.xlabel("Feature Importance Score")
plt.ylabel("Feature")
plt.title("Top 10 Important Features")
plt.gca().invert_yaxis()
plt.show()


In [ ]:
from sklearn.feature_selection import SelectFromModel
selector = SelectFromModel(model, threshold="median", prefit=True)
X_selected = selector.transform(X_train)


In [ ]:
X_train["Amount_per_V14"] = X_train["Amount"] / (X_train["V14"] + 1e-9)


In [ ]:
model = XGBClassifier(scale_pos_weight=scale_pos_weight)


In [ ]:
from sklearn.exceptions import NotFittedError

try:
    y_pred_new = (model.predict_proba(X_test)[:, 1] >= 0.3).astype(int)
except NotFittedError:
    print("Model is not trained! Retraining now...")
    model.fit(X_train, y_train)
    y_pred_new = (model.predict_proba(X_test)[:, 1] >= 0.3).astype(int)


In [ ]:
new_threshold = 0.3  # Instead of 0.5
y_pred_new = (model.predict_proba(X_test)[:, 1] >= new_threshold).astype(int)
